In [13]:
#Inherited from https://www.kaggle.com/j105sahil/eeg-brainwave-dataset-mental-state

from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [14]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

import xgboost as xgb

import warnings
warnings.filterwarnings('ignore')

# import os
# os.listdir('../input')

In [58]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
# #Script to convert from mean centered to 0 - 1600 uv
# data = res[1:]
# data = 0.48828125 * (np.array(data))
# 1680 * ((data) - data.min()) / (data.max() - data.min())

In [183]:
!mkdir dataset/transformed_data

In [185]:
#Inherited from https://github.com/jordan-bird/eeg-feature-generation

# data transformation from 0 - 1600 to mean centering around 0
# https://github.com/alexandrebarachant/muse-lsl/issues/11


for x in os.listdir("dataset/original_data"):

    # Ignore non-CSV files
    if not x.lower().endswith('.csv'):
        continue
    print("processing ", x)
    df = pd.read_csv("dataset/original_data/"+x)
    for electrode in ["TP9", "TP10", "AF7", "AF8", "Right AUX"]:    
        data = df[electrode]
        data = 0.48828125 * (np.array(data))
        data = 1680 * ((data) - data.min()) / (data.max() - data.min())
        df[electrode] = data
    df.to_csv("dataset/transformed_data/"+x, index=False)

In [186]:


from EEG_generate_training_matrix import gen_training_matrix

data_directory_path = "dataset/transformed_data/"
preprocessed_data_file_name = "out.csv"

gen_training_matrix(data_directory_path, preprocessed_data_file_name, cols_to_ignore = -1)

Using file subjectb-relaxed-2.csv
resulting vector shape for the file (6, 989)
Using file subjectb-concentrating-1.csv
resulting vector shape for the file (86, 989)
Using file subjectc-neutral-2.csv
resulting vector shape for the file (16, 989)
Using file subjecta-concentrating-2.csv
resulting vector shape for the file (102, 989)
Using file subjectc-concentrating-1.csv
resulting vector shape for the file (116, 989)
Using file subjectb-relaxed-1.csv
resulting vector shape for the file (116, 989)
Using file subjectb-concentrating-2.csv
resulting vector shape for the file (87, 989)
Using file subjectc-neutral-1.csv
resulting vector shape for the file (116, 989)
Using file subjecta-concentrating-1.csv
resulting vector shape for the file (116, 989)
Using file subjectc-concentrating-2.csv
resulting vector shape for the file (116, 989)
Using file subjecta-relaxed-1.csv
resulting vector shape for the file (116, 989)
Using file subjectd-neutral-1.csv
resulting vector shape for the file (116, 98

In [187]:
nRowsRead = None # specify 'None' if want to read whole file
# mental-state.csv has 2360 rows in reality, but we are only loading/previewing the first 1000 rows
df = pd.read_csv(preprocessed_data_file_name, delimiter=',', nrows = nRowsRead)
df.dataframeName = preprocessed_data_file_name
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')
df

There are 2479 rows and 989 columns


,lag1_mean_0,lag1_mean_1,lag1_mean_2,lag1_mean_3,lag1_mean_d_h2h1_0,lag1_mean_d_h2h1_1,lag1_mean_d_h2h1_2,lag1_mean_d_h2h1_3,lag1_mean_q1_0,lag1_mean_q1_1,...,freq_669_3,freq_679_3,freq_689_3,freq_699_3,freq_709_3,freq_720_3,freq_730_3,freq_740_3,freq_750_3,Label
0,921.471,789.770,975.471,897.938,26.465,80.836,34.338,-10.073,873.271,711.328,...,0.005,0.009,0.017,0.006,0.003,0.010,0.008,0.008,0.015,2.000
1,515.449,804.758,975.718,626.218,-51.633,-126.589,162.958,0.115,540.817,920.374,...,0.003,0.006,0.012,0.003,0.010,0.013,0.006,0.005,0.005,0.000
2,420.767,753.867,906.633,499.492,82.219,76.523,47.940,67.416,371.036,719.579,...,0.012,0.006,0.007,0.005,0.003,0.009,0.007,0.012,0.003,0.000
3,657.403,1473.209,674.606,723.010,-52.555,16.960,223.668,-116.777,666.154,1451.359,...,0.008,0.004,0.008,0.001,0.002,0.009,0.007,0.008,0.008,0.000
4,1015.545,706.362,507.026,1027.509,0.858,-11.982,-97.209,-23.571,1039.092,709.531,...,0.006,0.006,0.008,0.018,0.003,0.004,0.003,0.002,0.010,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2474,965.036,826.032,940.396,937.369,-41.855,-33.397,-133.594,-51.514,1016.081,833.538,...,0.001,0.007,0.004,0.005,0.009,0.007,0.007,0.005,0.004,2.000
2475,1148.750,857.038,846.509,1063.284,17.402,13.010,2.028,-34.279,1149.221,848.629,...,0.003,0.027,0.010,0.020,0.006,0.023,0.019,0.006,0.017,2.000
2476,1030.443,862.922,783.806,1130.396,-12.548,-36.101,-127.865,-31.576,1039.389,892.795,...,0.005,0.001,0.002,0.001,0.003,0.001,0.002,0.006,0.003,2.000
2477,1115.711,704.877,641.492,1117.700,-3.971,0.391,-19.994,-34.478,1135.100,684.976,...,0.005,0.001,0.002,0.007,0.005,0.002,0.006,0.003,0.004,2.000


In [188]:
df.head(5)

,lag1_mean_0,lag1_mean_1,lag1_mean_2,lag1_mean_3,lag1_mean_d_h2h1_0,lag1_mean_d_h2h1_1,lag1_mean_d_h2h1_2,lag1_mean_d_h2h1_3,lag1_mean_q1_0,lag1_mean_q1_1,...,freq_669_3,freq_679_3,freq_689_3,freq_699_3,freq_709_3,freq_720_3,freq_730_3,freq_740_3,freq_750_3,Label
0,921.471,789.770,975.471,897.938,26.465,80.836,34.338,-10.073,873.271,711.328,...,0.005,0.009,0.017,0.006,0.003,0.010,0.008,0.008,0.015,2.000
1,515.449,804.758,975.718,626.218,-51.633,-126.589,162.958,0.115,540.817,920.374,...,0.003,0.006,0.012,0.003,0.010,0.013,0.006,0.005,0.005,0.000
2,420.767,753.867,906.633,499.492,82.219,76.523,47.940,67.416,371.036,719.579,...,0.012,0.006,0.007,0.005,0.003,0.009,0.007,0.012,0.003,0.000
3,657.403,1473.209,674.606,723.010,-52.555,16.960,223.668,-116.777,666.154,1451.359,...,0.008,0.004,0.008,0.001,0.002,0.009,0.007,0.008,0.008,0.000
4,1015.545,706.362,507.026,1027.509,0.858,-11.982,-97.209,-23.571,1039.092,709.531,...,0.006,0.006,0.008,0.018,0.003,0.004,0.003,0.002,0.010,1.000


In [189]:
from sklearn import ensemble

model = ensemble.RandomForestClassifier(n_estimators=15,max_depth=4)

In [190]:
msk = np.random.rand(len(df)) < 0.7

train = df[msk]
test = df[~msk]

y_train = train["Label"]
y_test = test["Label"]

X_train = train.drop("Label", axis=1)
X_test = test.drop("Label", axis=1)

model.fit(X_train, y_train)

RandomForestClassifier(max_depth=4, n_estimators=15)

In [191]:
y_pred = model.predict(X_test)

In [244]:
from sklearn import metrics

count_misclassified = (y_test != y_pred).sum()
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = metrics.accuracy_score(y_test, y_pred)
print('Accuracy: {:.2f}'.format(accuracy))

Misclassified samples: 108
Accuracy: 0.85


In [245]:
y_pred

array([0., 1., 0., 2., 0., 2., 2., 2., 0., 2., 2., 1., 1., 1., 1., 2., 2.,
       1., 0., 1., 0., 1., 2., 1., 2., 0., 1., 0., 2., 1., 2., 2., 1., 1.,
       1., 1., 2., 0., 2., 1., 0., 2., 2., 0., 0., 1., 2., 0., 2., 1., 1.,
       2., 0., 1., 2., 0., 2., 1., 1., 1., 0., 2., 1., 2., 1., 1., 1., 0.,
       2., 0., 2., 1., 2., 2., 1., 0., 2., 0., 1., 1., 1., 1., 1., 0., 1.,
       1., 0., 2., 2., 0., 0., 1., 1., 1., 2., 1., 1., 1., 2., 0., 2., 1.,
       0., 2., 1., 2., 1., 0., 1., 0., 1., 1., 2., 1., 1., 0., 1., 2., 2.,
       1., 0., 0., 1., 0., 0., 0., 0., 2., 1., 1., 0., 0., 0., 2., 1., 1.,
       0., 2., 1., 1., 1., 2., 0., 2., 2., 0., 0., 1., 1., 1., 2., 1., 1.,
       2., 2., 2., 2., 0., 0., 2., 2., 1., 1., 1., 0., 1., 0., 0., 0., 1.,
       0., 0., 0., 1., 0., 1., 1., 2., 1., 0., 2., 2., 2., 1., 1., 1., 0.,
       2., 1., 1., 1., 1., 2., 1., 0., 1., 1., 2., 1., 0., 0., 0., 2., 2.,
       0., 2., 2., 2., 1., 2., 2., 1., 1., 1., 2., 1., 0., 1., 1., 0., 1.,
       1., 2., 2., 1., 0.

### Testing on muse monitor converted data

In [239]:
nRowsRead = None # specify 'None' if want to read whole file
# mental-state.csv has 2360 rows in reality, but we are only loading/previewing the first 1000 rows
preprocessed_data_file_name = "out_testing.csv"
df = pd.read_csv(preprocessed_data_file_name, delimiter=',', nrows = nRowsRead)
df.dataframeName = preprocessed_data_file_name
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')
df

There are 1072 rows and 989 columns


,lag1_mean_0,lag1_mean_1,lag1_mean_2,lag1_mean_3,lag1_mean_d_h2h1_0,lag1_mean_d_h2h1_1,lag1_mean_d_h2h1_2,lag1_mean_d_h2h1_3,lag1_mean_q1_0,lag1_mean_q1_1,...,freq_669_3,freq_679_3,freq_689_3,freq_699_3,freq_709_3,freq_720_3,freq_730_3,freq_740_3,freq_750_3,Label
0,803.329,804.456,805.762,804.614,-13.661,-5.072,2.149,-14.104,799.988,805.709,...,0.020,0.016,0.011,0.012,0.015,0.009,0.011,0.017,0.004,2.000
1,803.563,805.615,805.260,806.155,-6.685,4.992,-0.706,-8.089,799.074,804.240,...,0.019,0.025,0.013,0.006,0.008,0.011,0.010,0.004,0.027,2.000
2,803.628,806.789,796.698,806.028,2.729,-0.436,-3.360,2.558,803.496,808.147,...,0.016,0.022,0.023,0.012,0.020,0.016,0.015,0.007,0.022,2.000
3,799.252,799.028,806.605,801.388,-11.970,-1.358,-6.542,-15.084,819.014,801.020,...,0.018,0.023,0.007,0.011,0.010,0.011,0.014,0.014,0.011,2.000
4,866.258,863.628,868.202,866.309,-78.630,-65.371,-53.748,-53.097,975.256,958.946,...,0.014,0.014,0.010,0.003,0.009,0.008,0.002,0.015,0.008,2.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1067,800.177,804.359,802.449,805.067,-11.726,5.269,-1.249,-25.614,805.061,803.167,...,0.031,0.022,0.020,0.009,0.006,0.015,0.006,0.033,0.021,2.000
1068,804.134,806.737,804.261,806.458,8.902,2.076,3.673,8.926,799.646,804.508,...,0.015,0.008,0.009,0.005,0.008,0.001,0.005,0.006,0.006,2.000
1069,797.217,802.635,803.620,800.074,-2.330,-6.274,-0.525,-2.135,800.297,808.660,...,0.029,0.019,0.020,0.033,0.025,0.014,0.009,0.002,0.007,2.000
1070,804.163,812.713,805.394,805.061,22.620,34.080,-14.478,0.186,791.093,778.553,...,0.001,0.007,0.001,0.006,0.003,0.005,0.003,0.003,0.002,2.000


In [240]:
X_muse_test = df.drop("Label", axis=1)

In [241]:
y_muse_pred = model.predict_proba(X_muse_test)

In [242]:
#debbie league part 1
np.sum(y_muse_pred, axis=0)

array([341.0599802, 408.5203079, 322.4197119])

In [238]:
#eyes closed
np.sum(y_muse_pred, axis=0)

array([34.85876267, 44.79209561, 30.34914172])

In [234]:
#eyes open 250hz
np.sum(y_muse_pred, axis=0)

array([5.31997684, 8.35767198, 5.32235118])

In [230]:
#eyes open relaxed
np.sum(y_muse_pred, axis=0)

array([42.18621703, 62.64924141, 50.16454156])

In [226]:
#league part 2
np.sum(y_muse_pred, axis=0)

array([135.15533979, 147.3246686 , 129.51999161])

In [220]:
(unique, counts) = np.unique(y_muse_pred, return_counts=True)
frequencies = np.asarray((unique, counts)).T

In [215]:
#eyes closed 
frequencies

array([[ 0., 10.],
       [ 1., 89.],
       [ 2., 11.]])

In [209]:
#eyes open relaxed
frequencies

array([[ 0., 37.],
       [ 1., 89.],
       [ 2., 29.]])

In [203]:
#league part 1
frequencies

array([[  0., 311.],
       [  1., 641.],
       [  2., 120.]])

In [221]:
#league part 2
frequencies

array([[  0., 158.],
       [  1., 181.],
       [  2.,  73.]])

## CNN and statistical features

### Paper: Classification of EEG Signals Based on Image Representation of Statistical Features
#### Link: https://link.springer.com/chapter/10.1007/978-3-030-29933-0_37

In [246]:
importances = model.feature_importances_

In [247]:
ranked = np.argsort(importances)[:729]

In [248]:
feature_imp_df = df[df.columns[:-1][ranked]] 
feature_imp_df["Label"] = df["Label"]

In [249]:
from sklearn import ensemble

model = ensemble.RandomForestClassifier(n_estimators=15,max_depth=4)

In [250]:
df = feature_imp_df

msk = np.random.rand(len(df)) < 0.7

train = df[msk]
test = df[~msk]

y_train = train["Label"]
y_test = test["Label"]

X_train = train.drop("Label", axis=1)
X_test = test.drop("Label", axis=1)

model.fit(X_train, y_train)

RandomForestClassifier(max_depth=4, n_estimators=15)

In [251]:
y_pred = model.predict(X_test)

In [252]:
from sklearn import metrics

count_misclassified = (y_test != y_pred).sum()
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = metrics.accuracy_score(y_test, y_pred)
print('Accuracy: {:.2f}'.format(accuracy))

Misclassified samples: 0
Accuracy: 1.00


In [253]:
X_test.shape

(326, 729)

In [255]:
X_train_np = X_train.to_numpy()
y_train_np = y_train.to_numpy()

X_test_np = X_test.to_numpy()
y_test_np = y_test.to_numpy()

In [256]:
X_train_np = X_train_np.reshape(X_train_np.shape[0], 27, 27, 1)
X_test_np = X_test_np.reshape(X_test_np.shape[0], 27, 27, 1)

In [257]:
X_train_np.shape

(746, 27, 27, 1)

In [258]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten
import matplotlib.pyplot as plt

In [259]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(27, 27, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(3))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [260]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(X_train_np, y_train_np, epochs=20, 
                    validation_data=(X_test_np, y_test_np))

Train on 746 samples, validate on 326 samples
Epoch 1/20
746/746 [==============================] - 1s 2ms/sample - loss: 2.3129 - acc: 0.9571 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 2/20
746/746 [==============================] - 1s 1ms/sample - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 3/20
746/746 [==============================] - 1s 1ms/sample - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 4/20
746/746 [==============================] - 1s 1ms/sample - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 5/20
746/746 [==============================] - 1s 1ms/sample - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 6/20
746/746 [==============================] - 1s 895us/sample - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 7/20
746/746 [==============================] - 1s 831us/sample - loss: 0.0000e+00 - acc: 1.0000 -

KeyboardInterrupt: 

In [110]:
test_loss, test_acc = model.evaluate(X_test_np,  y_test_np, verbose=2)

697/697 - 0s - loss: 0.2234 - acc: 0.9383


In [121]:
model_preds = np.argmax(model.predict(X_test_np), axis=1)

In [123]:
correct_preds = np.sum(model_preds == y_test_np)
correct_preds

654